In [9]:
import pandas as pd
import itertools
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from xgboost import XGBRegressor

In [11]:
def evaluate_all_combinations_cv_xgb(X, y):
    results = []
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    for r in range(1, len(X.columns) + 1):
        for combo in itertools.combinations(X.columns, r):
            X_subset = X[list(combo)]
            model = XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42, verbosity=0)
            scores = cross_val_score(model, X_subset, y, cv=kf, scoring='r2')
            mean_r2 = scores.mean()
            results.append({
                'num_features': r,
                'features': combo,
                'mean_r2': mean_r2
            })

    results_df = pd.DataFrame(results).sort_values(by='mean_r2', ascending=False).reset_index(drop=True)

    for i, row in results_df.iterrows():
        print(f"{i+1}. Features: {row['features']}, R²: {row['mean_r2']:.4f}")

    best = results_df.iloc[0]
    return best['features'], best['mean_r2'], results_df

In [12]:
df = pd.read_csv('dataset.csv')
X = df[['cement', 'blast furnace slag', 'fly ash', 'water ', 'superplasticizer', 'coarse aggregate', 'fine aggregate', 'age']]
y = df['CCS']

# XGBoost로 조합 평가
best_combo, best_r2, all_results = evaluate_all_combinations_cv_xgb(X, y)

print("\n📌 최고 조합:", best_combo)
print("📈 최고 평균 결정계수(R²):", best_r2)

1. Features: ('cement', 'blast furnace slag', 'fly ash', 'water ', 'superplasticizer', 'coarse aggregate', 'fine aggregate', 'age'), R²: 0.9030
2. Features: ('cement', 'blast furnace slag', 'fly ash', 'water ', 'superplasticizer', 'fine aggregate', 'age'), R²: 0.9030
3. Features: ('cement', 'blast furnace slag', 'water ', 'superplasticizer', 'coarse aggregate', 'fine aggregate', 'age'), R²: 0.9023
4. Features: ('cement', 'blast furnace slag', 'fly ash', 'water ', 'coarse aggregate', 'fine aggregate', 'age'), R²: 0.9007
5. Features: ('cement', 'blast furnace slag', 'fly ash', 'water ', 'fine aggregate', 'age'), R²: 0.9007
6. Features: ('cement', 'blast furnace slag', 'water ', 'superplasticizer', 'fine aggregate', 'age'), R²: 0.9005
7. Features: ('cement', 'blast furnace slag', 'water ', 'coarse aggregate', 'fine aggregate', 'age'), R²: 0.9001
8. Features: ('cement', 'blast furnace slag', 'water ', 'fine aggregate', 'age'), R²: 0.8975
9. Features: ('cement', 'blast furnace slag', 'fly a